In [ ]:
def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0]) + contoursColors[q]['x'] 
                        y = int(j[1]) + contoursColors[q]['y'] 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) + contoursColors[q]['x'] 
                                    y = int(j[1]) + contoursColors[q]['y'] 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'model1+model2', 
        "description": 'model1+model2',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos

import os

def main():
    parser = argparse.ArgumentParser(
                        prog='DSA FTU Annotations to Patches',
                        description='Extract DSA FTU Annotations to Patches',
                        epilog='Text at the bottom of help')

    parser.add_argument('--svsBase', help='/blue/pinaki.sarder/...')     
    parser.add_argument('--fid', help='folder id in DSA')     
    parser.add_argument('--outputdir', help='/orange/pinaki.sarder/ahmed.naglah/...')     
    parser.add_argument('--username', help='username')     
    parser.add_argument('--password', help='password')
    parser.add_argument('--patchSize', help='patchSize', type=int, default=512)
    parser.add_argument('--thresholdArea', help='thresholdArea', type=float, default=0.3)
    parser.add_argument('--spatialResolution', help='spatialResolution', type=float, default=0.25)      
    parser.add_argument('--apiUrl', help='https://athena.rc.ufl.edu/api/v1')   
    parser.add_argument('--name', help='a name for the pipeline', default='defaultNaglahPipeline')     
    parser.add_argument('--layerName', help='annotation layer on DSA', type=str)

    args = parser.parse_args()

    config = vars(args)

    outdir = config['outputdir']

    if not os.path.exists(outdir):
        os.mkdir(outdir)

    dsa = DSA(config)

    dsaFolder = DSAFolder(config)
    items = dsaFolder.items

    for i in range(len(items)):
        fid, svsname = items[i]

        try:

            item = DSAItem(config, fid, svsname)
            anno = item.annotations

            if len(anno)>0:
                slide = Slide(config, svsname)

                slide.extractForeground()

                slide.createBinary(anno)

                patches = slide.extractPairs(config['patchSize'])

                print(len(patches))

                contoursColors = []
                for key in patches:
                    try:
                        patch = patches[key]
                        contours = segmentTBMPair(patch)
                        x, y = key.split('_')
                        contoursColors.append({'x': int(x), 'y': int(y) , 'cnt': contours, 'color': '255, 20, 20'})
                        cv2.imwrite(f"{outdir}/{key}.png", patch)
                    except:
                        pass
                annos = generateAnno(contoursColors)
                dsa.postAnno(fid, annos)
            else:
                logging.warning(f"No annotation found in {fid} __ {svsname}")
        except:
            logging.warning(f"Exeption in {fid} __ {svsname}")

In [ ]:
import cv2
import numpy as np
from modules.ftu import Tubule
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'model1+model2', 
        "description": 'model1+model2',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos



def getContours(mask):

    def smoothBinary(mask2d):
        kn = 2
        iterat = 2
        kernel = np.ones((kn, kn), np.uint8) 
        for _ in range(iterat):
            mask2d = cv2.erode(mask2d, kernel, iterations=1) 
            mask2d = cv2.dilate(mask2d, kernel, iterations=1) 
        return mask2d
    
    mask2 = cv2.cvtColor(mask, cv2.BGR2GRAY)
    
    maskSmooth = smoothBinary(mask2)

    thresholdArea = 0.0001

    contours, hierarchy = cv2.findContours(maskSmooth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    c_max = max(contours, key = cv2.contourArea)
    area_min = cv2.contourArea(c_max) * thresholdArea

    contours_filtered = []

    amin = 100

    for contour, hie in zip(contours, hierarchy[0]):
        area = cv2.contourArea(contour)
        if area > area_min:
            if area > amin:
                msk_ = mask*0
                interThickness = 2
                cv2.drawContours(msk_, [contour], -1, [255,255,255], interThickness)
                if np.sum(msk_*mask)>0:
                    contours_filtered.append((contour,hie))

    return contours_filtered

xx = 95663
yy = 41940


mask = np.zeros((yy, xx, 3), dtype='uint8')

mydir = "/orange/pinaki.sarder/ahmed.naglah/data/tbm_dataset_split2/pred"

files = os.listdir(mydir)

for file in files:
    lst = file.split('[')[-1].split(']')[0].replace("=", "").replace("x", "").replace("w", "").replace("y", "").replace("h", "").split(",")
    print(lst)
    x, y, w, h = [int(k) for k in lst]
    mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)

contours = getContours(mask)
contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

dsafid = {
    '000042_PAS_2of2':'66100f31d2f45e99a94d55dd',
    '004716_PAS_2of2':'66100f94d2f45e99a94d5b47',
    '008283_PAS_2of2':'66100ac4d2f45e99a94d4299',
    '009781_PAS_2of2':'66100848d2f45e99a94d1779',
    '009782_PAS_1of2':'661008b1d2f45e99a94d194b',
    '014312_PAS_1of1':'66100dbfd2f45e99a94d4cbf',
    '019412_PAS_2of2':'661008f4d2f45e99a94d19e4',
    '019413_PAS_1of2':'6610094bd2f45e99a94d20eb'
}

slides2 = [
    '009781_PAS_2of2'
]

annoooos = {}

for name in slides2:
    config = {
        'fid': dsafid[name],
        'username': '',
        'password': '',
        'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
        'layerName': 'tbm_geojson',
        'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
    }

    annos = generateAnno(contoursColors)

    annoooos[name] = annos

    dsa = DSA(config)

    fid = dsafid[name]

    annos = annoooos[name]
    dsa.postAnno(fid, annos)

In [ ]:
import cv2
import numpy as np
from modules.ftu import Tubule
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            #print(hie)
            #print(cnts[f])
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'ai_model_2', 
        "description": 'ai_model_2',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos



def getContours(mask):

    def smoothBinary(mask2d):
        kn = 2
        iterat = 2
        kernel = np.ones((kn, kn), np.uint8) 
        for _ in range(iterat):
            mask2d = cv2.erode(mask2d, kernel, iterations=1) 
            mask2d = cv2.dilate(mask2d, kernel, iterations=1) 
        return mask2d
    
    mask2 = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    maskSmooth = smoothBinary(mask2)

    thresholdArea = 0.0001

    contours, hierarchy = cv2.findContours(maskSmooth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #c_max = max(contours, key = cv2.contourArea)
    #area_min = cv2.contourArea(c_max) * thresholdArea

    contours_filtered = contours

    #amin = 100

    #for contour, hie in zip(contours, hierarchy[0]):
    #    area = cv2.contourArea(contour)
    #    if area > area_min:
    #        if area > amin:
    #            msk_ = mask*0
    #            interThickness = 2
    #            cv2.drawContours(msk_, [contour], -1, [255,255,255], interThickness)
    #            if np.sum(msk_*mask)>0:
    #                contours_filtered.append((contour,hie))
    
    print(len(contours))
    print(len(hierarchy[0]))
    
    return [(contours[q], hierarchy[0][q]) for q in range(len(contours))]

xx = 129544
yy = 100000

mask = np.zeros((yy, xx, 3), dtype='uint8')

# Prediction Directories with each slide as subdirectory ... 
# rootdir = "/orange/pinaki.sarder/ahmed.naglah/data/tbm_dataset_split2/pred"
# Image Format 77312x_23040y.jpg predicted binary mask
rootdir = "/blue/pinaki.sarder/h.lohaan/KPMP/segnet_predictions_for_10_slides"

ssss_slides = [
    'S-1909-007149_PAS_1of2',
    'S-1905-018731_PAS_2of2',
    'S-1908-010066_PAS_1of2',
    '18-162_PAS_4of6',
    'S-1905-017738_PAS_1of2',
    'S-2103-004857_PAS_2of2',
    'S-2106-003588_PAS_1of2',
    'S-1910-000089_PAS_2of2',
    'S-2001-005357_PAS_1of2',
    'S-1904-007293_PAS_1of2',
]

dsafid = {
    '000042_PAS_2of2':'66100f31d2f45e99a94d55dd',
    '004716_PAS_2of2':'66100f94d2f45e99a94d5b47',
    '008283_PAS_2of2':'66100ac4d2f45e99a94d4299',
    '009781_PAS_2of2':'66100848d2f45e99a94d1779',
    '009782_PAS_1of2':'661008b1d2f45e99a94d194b',
    '014312_PAS_1of1':'66100dbfd2f45e99a94d4cbf',
    '019412_PAS_2of2':'661008f4d2f45e99a94d19e4',
    '019413_PAS_1of2':'6610094bd2f45e99a94d20eb',
    'S-1909-007149_PAS_1of2':'66918e4a0519743b266cee1b',
    'S-1905-018731_PAS_2of2': '66946f630519743b266f75fa',
    'S-1908-010066_PAS_1of2': '669473920519743b26705428',
    '18-162_PAS_4of6':'6694760a0519743b2670d8e0',
    'S-1905-017738_PAS_1of2':'66946ed40519743b266f75ee',
    'S-2103-004857_PAS_2of2': '66951e5a0519743b267b8fa8',
    'S-2106-003588_PAS_1of2': '66951e580519743b267b8fa5',
    'S-1910-000089_PAS_2of2':  '66951fba0519743b267b8fea',
    'S-2001-005357_PAS_1of2': '66951e5b0519743b267b8fab',
    'S-1904-007293_PAS_1of2': '669520ca0519743b267cfe0a',
}

for ssss in ssss_slides:
    # ssss = "S-1904-007293_PAS_1of2"

    print(ssss)

    mydir = f"{rootdir}/{ssss}"

    import os

    files = os.listdir(mydir)

    print(len(files))

    for file in files:
        #print(file)
        try:
            lst = file.replace('y.jpg', '').replace('x','').split('_')
            #print(lst)

            w = 512
            h = 512

            x, y = [int(k) for k in lst]
            mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)
        except:
            pass

    print('processing contours')
    contours = getContours(mask)

    print('saving')
    contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

    slides2 = [
        ssss
    ]

    annoooos = {}

    for name in slides2:
        config = {
            'fid': dsafid[name],
            'username': 'ahmednaglah',
            'password': '',
            'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
            'layerName': 'tbm_hl_ai',
            'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
        }

        annos = generateAnno(contoursColors)

        annoooos[name] = annos

        dsa = DSA(config)

        fid = dsafid[name]

        annos = annoooos[name]
        dsa.postAnno(fid, annos)